<a href="https://colab.research.google.com/github/prisha90/Study-Oriented-Project-/blob/main/Glaucoma_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import numpy as np
import pandas as pd
import cv2
import tensorflow as tf
from tensorflow.keras import layers, models
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.ensemble import AdaBoostClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import GridSearchCV
from skimage import measure, morphology

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
IMAGES_FOLDER = "/content/drive/MyDrive/SOP (Rajya Laksmi Ma'am)/Work Document/G1020/Images"
MASK_FOLDER = "/content/drive/MyDrive/SOP (Rajya Laksmi Ma'am)/Work Document/G1020/Masks"
df = pd.read_csv("G1020.csv")

In [ ]:
def augment_image(image):
    """Apply data augmentation (rotation, flipping, contrast)."""
    angle = np.random.randint(-10, 10)
    rotated = cv2.warpAffine(image, cv2.getRotationMatrix2D((128, 128), angle, 1), (256, 256))
    flipped = cv2.flip(rotated, np.random.choice([-1, 0, 1]))
    alpha = np.random.uniform(0.8, 1.2)
    beta = np.random.randint(-30, 30)
    return cv2.convertScaleAbs(flipped, alpha=alpha, beta=beta)


In [ ]:
images, labels, od_masks, oc_masks = [], [], [], []
image_size = (256, 256)

print(f"Total images in dataframe: {len(df)}")

if not os.path.exists(IMAGES_FOLDER):
    print(f"Error: IMAGES_FOLDER not found: {IMAGES_FOLDER}")
else:
    print(f"IMAGES_FOLDER found: {IMAGES_FOLDER}")

missing_images = [img for img in df['imageID'] if not os.path.exists(os.path.join(IMAGES_FOLDER, img))]
if missing_images:
    print(f"Missing images: {len(missing_images)} images not found.")



In [ ]:
for index, row in df.iterrows():
    img_path = os.path.join(IMAGES_FOLDER, row['imageID'])
    od_path = os.path.join(MASK_FOLDER, row['imageID'].replace(".jpg", "_od.png"))
    oc_path = os.path.join(MASK_FOLDER, row['imageID'].replace(".jpg", "_oc.png"))
    label = row['binaryLabels']

    if os.path.exists(img_path) and os.path.exists(od_path) and os.path.exists(oc_path):
        img = cv2.imread(img_path)
        img = cv2.resize(img, image_size)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        img = img.reshape(image_size[0], image_size[1], 1)

        od_mask = cv2.imread(od_path, cv2.IMREAD_GRAYSCALE)
        oc_mask = cv2.imread(oc_path, cv2.IMREAD_GRAYSCALE)

        images.append(augment_image(img))
        labels.append(label)
        od_masks.append(od_mask)
        oc_masks.append(oc_mask)

In [ ]:
images = np.array(images) / 255.0
od_masks = np.array(od_masks) / 255.0
oc_masks = np.array(oc_masks) / 255.0
labels = np.array(labels)

In [ ]:
X_train, X_temp, od_train, od_temp, oc_train, oc_temp, y_train, y_temp = train_test_split(
    images, od_masks, oc_masks, labels, test_size=0.3, random_state=42)
X_val, X_test, od_val, od_test, oc_val, oc_test, y_val, y_test = train_test_split(
    X_temp, od_temp, oc_temp, y_temp, test_size=0.5, random_state=42)


In [ ]:
# Optic disc and optic cup segmentation
def build_unet():
    inputs = layers.Input((256, 256, 1))
    c1 = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(inputs)
    p1 = layers.MaxPooling2D((2, 2))(c1)

    c2 = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(p1)
    p2 = layers.MaxPooling2D((2, 2))(c2)

    c3 = layers.Conv2D(256, (3, 3), activation='relu', padding='same')(p2)

    u1 = layers.UpSampling2D((2, 2))(c3)
    d1 = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(u1)

    u2 = layers.UpSampling2D((2, 2))(d1)
    d2 = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(u2)

    od_output = layers.Conv2D(1, (1, 1), activation='sigmoid', name='optic_disc')(d2)
    oc_output = layers.Conv2D(1, (1, 1), activation='sigmoid', name='optic_cup')(d2)

    return models.Model(inputs, [od_output, oc_output])

In [ ]:
unet = build_unet()
unet.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
unet.fit(X_train, [od_train, oc_train], epochs=15, batch_size=8, validation_data=(X_val, [od_val, oc_val]))


In [ ]:
od_pred, oc_pred = unet.predict(X_test)

In [ ]:
def refine_mask(mask):
    """Apply morphological operations to clean segmentation masks."""
    mask = (mask > 0.5).astype(np.uint8)  # Convert to binary
    mask = morphology.remove_small_objects(mask.astype(bool), min_size=500)
    mask = morphology.remove_small_holes(mask.astype(bool), area_threshold=500)
    return mask.astype(np.uint8)

od_refined = np.array([refine_mask(mask) for mask in od_pred])
oc_refined = np.array([refine_mask(mask) for mask in oc_pred])

In [ ]:
def calculate_cdr(disc_mask, cup_mask):
    disc_area = np.sum(disc_mask)
    cup_area = np.sum(cup_mask)
    return round((2 * cup_area) / disc_area, 4) if disc_area != 0 else 0

cdr_test = np.array([calculate_cdr(od, oc) for od, oc in zip(od_pred, oc_pred)])


def extract_isnt_quadrants(disc_mask, cup_mask):
    """Compute ISNT Ratio from Neuroretinal Rim (NRR)."""
    nrr_mask = cv2.bitwise_xor(disc_mask, cup_mask)

    height, width = disc_mask.shape
    cX, cY = width // 2, height // 2

    I = np.sum(nrr_mask[cY:, :])  # Inferior
    S = np.sum(nrr_mask[:cY, :])  # Superior
    N = np.sum(nrr_mask[:, :cX])  # Nasal
    T = np.sum(nrr_mask[:, cX:])  # Temporal

    # Compute ISNT Ratio
    if (N + T) == 0:
        return 0  # Avoid division by zero
    isnt_ratio = round((1 + (I + S)) / (1 + (N + T)), 4)
    return isnt_ratio


def extract_blood_vessels(fundus_image):
    """Extract blood vessels using CLAHE and bottom-hat filtering."""
    # Convert to grayscale
    green_channel = fundus_image[:, :, 1]  # Extract green channel (better contrast)

    # Apply CLAHE for contrast enhancement
    clahe = cv2.createCLAHE(clipLimit=4.0, tileGridSize=(6,6))
    enhanced = clahe.apply(green_channel)

    # Bottom-hat filtering to extract vessels
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (15, 15))
    bottom_hat = cv2.morphologyEx(enhanced, cv2.MORPH_BLACKHAT, kernel)

    # Thresholding for binary segmentation
    threshold_value = 3.15 * np.std(bottom_hat)  # Adaptive threshold
    _, vessel_mask = cv2.threshold(bottom_hat, threshold_value, 255, cv2.THRESH_BINARY)

    # Compute ISNT quadrants for blood vessels
    height, width = vessel_mask.shape
    I = np.sum(vessel_mask[height//2:, :])  # Inferior
    S = np.sum(vessel_mask[:height//2, :])  # Superior
    N = np.sum(vessel_mask[:, :width//2])  # Nasal
    T = np.sum(vessel_mask[:, width//2:])  # Temporal

    if (N + T) == 0:
        return 0  # Avoid division by zero
    blood_vessel_ratio = round((1 + (I + S)) / (1 + (N + T)), 4)
    return blood_vessel_ratio


In [ ]:
cdr_train = np.array([calculate_cdr(od, oc) for od, oc in zip(od_pred, oc_pred)])
isnt_train = np.array([extract_isnt_quadrants(od, oc) for od, oc in zip(od_pred, oc_pred)])
bvr_train = np.array([extract_blood_vessels(img) for img in X_train])

cdr_test = np.array([calculate_cdr(od, oc) for od, oc in zip(od_test, oc_test)])
isnt_test = np.array([extract_isnt_quadrants(od, oc) for od, oc in zip(od_test, oc_test)])
bvr_test = np.array([extract_blood_vessels(img) for img in X_test])


In [ ]:
X_train_combined = np.hstack((cdr_train.reshape(-1,1), isnt_train.reshape(-1,1), bvr_train.reshape(-1,1)))
X_test_combined = np.hstack((cdr_test.reshape(-1,1), isnt_test.reshape(-1,1), bvr_test.reshape(-1,1)))


In [ ]:
svm = SVC(kernel='rbf')
mlp = MLPClassifier(hidden_layer_sizes=(100,), max_iter=500)
adaboost = AdaBoostClassifier(n_estimators=50)

# Hyperparameter Tuning for SVM
param_grid = {'C': [0.1, 1, 10], 'gamma': [0.01, 0.1, 1]}
grid_search = GridSearchCV(SVC(kernel='rbf'), param_grid, cv=3)
grid_search.fit(cdr_test.reshape(-1, 1), y_test)

# Train models on extracted features
svm.fit(X_train_combined, y_train)
mlp.fit(X_train_combined, y_train)
adaboost.fit(X_train_combined, y_train)

In [ ]:
y_pred_svm = svm.predict(X_test_combined)
y_pred_mlp = mlp.predict(X_test_combined)
y_pred_adaboost = adaboost.predict(X_test_combined)


In [ ]:
print("SVM Accuracy:", accuracy_score(y_test, y_pred_svm))
print("MLP Accuracy:", accuracy_score(y_test, y_pred_mlp))
print("AdaBoost Accuracy:", accuracy_score(y_test, y_pred_adaboost))
print("Best SVM Parameters:", grid_search.best_params_)